# ntopconf 2023 - takeaway #2

### Creating a csv file containing a dataset with legit and suspicious domains


In [ ]:
import os
import json

from ntopng.ntopng import Ntopng

username     = "admin"
password     = "admin1"
ntopng_url   = "http://localhost:3000"

suspicious_dom = open("suspicious_domains.txt","w")

try:
	my_ntopng = Ntopng(username, password, None, ntopng_url)
	interfaces_available = my_ntopng.get_interfaces_list()
	#for interface in interfaces_available:
	#	print (interface)

	# {'ifid': 1, 'name': 'ens33', 'ifname': 'ens33'}
	# {'ifid': 2, 'name': 'lo', 'ifname': 'lo'}
	# OR
	# {'ifname': 'dns_tunnelling.pcapng', 'name': 'dns_tunnelling.pcapng', 'ifid': 3}

	my_interface = my_ntopng.get_interface(3)
	traffic = my_interface.get_active_flows_paginated(1,500)
	traffic_b = json.dumps(traffic, indent=4)
	#print (traffic_b)

	unique_info = []
	if 'data' in traffic:
		for entry in traffic['data']:
			if ('DNS' in entry['protocol']['l7'] and entry['info'] not in unique_info):
				unique_info.append(entry['info'])
				print(entry['info'])
				suspicious_dom.write(entry['info']+',1\n') # add 1
                
except ValueError as e:
	print(e)
	os._exit(-1)

#### Genereting legit domains

In [ ]:
file_subdomains = open('resources/most-common-subs-100.txt', 'r')
file_domains = open('resources/alexa-top100.txt', 'r')
file_legit_domains = open ('resources/legit_domains.txt', 'w')

subdomains = file_subdomains.readlines()
domains = file_domains.readlines()

for s1 in subdomains:
    for s2 in domains:
        file_legit_domains.write(s1.strip() + '.' + s2.strip() + ',0\n')
        print(s1.strip() + '.' + s2.strip())        
print ("Done")

#### Create a single file 'dataset.csv' with legit and suspicious domains

In [ ]:
files = ['resources/suspicious_domains.txt','resources/legit_domains.txt' ]

with open ('resources/dataset.txt', 'w') as dataset:
    for f in files:
        with open (f) as ifile:
            dataset.write(ifile.read())
print ("Done")